In [2]:
import numpy as np
np.random.seed(1001)
import os
import shutil
import pandas as pd
from util import *
from dataset import *
from cnn2d import *
from transform import *
import librosa


/home/kownse/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
train_root = '../data/audio_train/'
test_root = '../data/audio_test/'

nrows = 1
test_df = pd.read_csv("../data/sample_submission.csv", nrows = nrows)
train_df = pd.read_csv("../data/train.csv", nrows = nrows)

In [3]:
train_1 = pd.read_csv('../data/train_mel.csv')

In [4]:
train_1.shape

(56838, 398)

In [10]:
def argu_df(df, root, idx):
    callbacks = [
        LoadAudio(None),
        ChangeAmplitude(), 
        ChangeSpeedAndPitchAudio(), 
        StretchAudio(),
        TimeshiftAudio(),
        RandomAudioLength(),
    ]
    
    df_argu = pd.DataFrame()
    pbar = tqdm(total=len(df))
    for row in df.iterrows():
        file_path = train_root + row[1].fname
        data =  prepare_wav(file_path, callbacks)
        ID = row[1].fname[:row[1].fname.find('.')]
        save_file = '{}_arg{}.wav'.format(ID, idx)
        save_path = root + save_file
        #print(save_path)
        librosa.output.write_wav(save_path, data['samples'], data['sample_rate'])
        
        new_row = row[1].copy()
        new_row.fname = save_file
        #print()
        df_argu = df_argu.append(new_row, ignore_index=True)
        
        pbar.update(1)
    pbar.close()
    
    return df_argu

In [11]:
def argu_df_bulk(df, root, cnt):
    for i in range(cnt):
        df_argu = argu_df(train_df, train_root, i)
        df = df.append(df_argu, ignore_index=True)
        
    return df

In [12]:
df_argu_train = argu_df_bulk(train_df, train_root, 3)
df_argu_train.to_csv('../data/train_argu.csv', index=False)

100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


In [7]:
df_argu_train

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0.0
1,00044347_arg0.wav,Hi-hat,0.0
2,00044347_arg1.wav,Hi-hat,0.0
3,00044347_arg2.wav,Hi-hat,0.0


In [ ]:
x2[0][0][0]

In [ ]:
x3.shape